# Explore here

It's recommended to use this notebook for exploration purposes.

In [ ]:
import os
from bs4 import BeautifulSoup
import requests
import time
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns


Could not find price element. Displaying page structure...
<!DOCTYPE html>
<html dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <meta content="ie=edge" http-equiv="x-ua-compatible"/>
  <meta content="width=device-width, initial-scale=1, maximum-scale=1, user-scalable=no, shrink-to-fit=no" name="viewport"/>
  <link href="/manifest.json" rel="manifest"/>
  <meta content="https://coinmarketcap.com/currencies/xrp/" property="og:url"/>
  <link href="https://coinmarketcap.com/currencies/xrp/" rel="canonical"/>
  <link href="https://coinmarketcap.com/ar/currencies/xrp/" hreflang="ar" rel="alternate"/>
  <link href="https://coinmarketcap.com/bg/currencies/xrp/" hreflang="bg" rel="alternate"/>
  <link href="https://coinmarketcap.com/cs/currencies/xrp/" hreflang="cs" rel="alternate"/>
  <link href="https://coinmarketcap.com/da/currencies/xrp/" hreflang="da" rel="alternate"/>
  <link href="https://coinmarketcap.com/de/currencies/xrp/" hreflang="de" rel="alternate"/>
  <link href="https:

In [ ]:
import pandas as pd

# Scrape top 10 songs from Wikipedia Spotify streaming records
wiki_url = "https://en.wikipedia.org/wiki/List_of_Spotify_streaming_records"

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
}

try:
    response = requests.get(wiki_url, headers=headers, timeout=10)
    response.raise_for_status()
    
    # Parse the HTML
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Find tables on the page - Wikipedia pages usually have structured tables
    tables = soup.find_all('table', {'class': 'wikitable'})
    
    if tables:
        # The first table usually contains the top streamed songs
        table = tables[0]
        rows = table.find_all('tr')[1:]  # Skip header row
        
        songs_data = []
        for i, row in enumerate(rows[:10]):  # Get top 10
            cols = row.find_all('td')
            if len(cols) >= 3:
                # Extract song name, artist, and streams
                rank = i + 1
                song = cols[1].get_text(strip=True) if len(cols) > 1 else "N/A"
                artist = cols[2].get_text(strip=True) if len(cols) > 2 else "N/A"
                streams = cols[3].get_text(strip=True) if len(cols) > 3 else "N/A"
                
                songs_data.append({
                    'Rank': rank,
                    'Song': song,
                    'Artist': artist,
                    'Streams': streams
                })
        
        if songs_data:
            df = pd.DataFrame(songs_data)
            print("Top 10 Most Streamed Songs on Spotify:\n")
            print(df.to_string(index=False))
        else:
            print("Could not extract song data from the table.")
    else:
        print("No tables found on the page.")
        
except requests.exceptions.RequestException as e:
    print(f"Error fetching the page: {e}")
except Exception as e:
    print(f"Error parsing the page: {e}")

Top 10 Most Streamed Songs on Spotify:

 Rank                          Song Artist           Streams Date
    1                    The Weeknd  5.258  29 November 2019  [1]
    2                    Ed Sheeran  4.750    6 January 2017  [2]
    3             The Neighbourhood  4.351   3 December 2012  [3]
    4        The WeekndandDaft Punk  4.340 21 September 2016  [4]
    5                  Harry Styles  4.226      1 April 2022  [5]
    6                 Lewis Capaldi  4.210   8 November 2018  [6]
    7        Post MaloneandSwae Lee  4.105   18 October 2018  [7]
    8        DrakewithWizkidandKyla  4.010      5 April 2016  [8]
    9                    Ed Sheeran  3.811      3 March 2017  [9]
   10 The Kid LaroiandJustin Bieber  3.786       9 July 2021 [10]


In [10]:
# Clean up the data and import into SQLite (preserving original categories)
import re

# Clean the songs_data list - keep original category names
cleaned_songs = []
for song in songs_data:
    cleaned_song = {
        'Rank': int(song['Rank']),
        'Song': song['Song'].strip(),
        'Artist': song['Artist'].strip(),
        'Streams': re.sub(r'[^\d,]', '', song['Streams']).replace(',', '') if song['Streams'] != 'N/A' else None
    }
    cleaned_songs.append(cleaned_song)

# Create DataFrame with cleaned data
df_cleaned = pd.DataFrame(cleaned_songs)
print("Cleaned Data:")
print(df_cleaned.to_string(index=False))
print("\n" + "="*80 + "\n")

# Create or connect to SQLite database
db_path = 'spotify_songs.db'
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Drop existing table if it exists (optional, for clean slate)
cursor.execute('DROP TABLE IF EXISTS top_songs')

# Create the table
cursor.execute('''
    CREATE TABLE top_songs (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        Rank INTEGER UNIQUE,
        Song TEXT NOT NULL,
        Artist TEXT NOT NULL,
        Streams INTEGER
    )
''')

# Insert cleaned data into the table
for _, row in df_cleaned.iterrows():
    cursor.execute('''
        INSERT INTO top_songs (Rank, Song, Artist, Streams)
        VALUES (?, ?, ?, ?)
    ''', (row['Rank'], row['Song'], row['Artist'], int(row['Streams']) if row['Streams'] else None))

conn.commit()

# Display the data from the database
query_result = cursor.execute('SELECT * FROM top_songs ORDER BY Rank').fetchall()
print(f"✓ Successfully imported {len(query_result)} songs into SQLite database\n")
print(f"Database: {db_path}\n")
print("Top 10 Spotify Songs (from database):")
print("-" * 80)
for row in query_result:
    song_id, rank, song, artist, streams = row
    streams_formatted = f"{streams:,}" if streams else "N/A"
    print(f"{rank:2}. {song:<40} | {artist:<20} | {streams_formatted:>12}")

conn.close()

Cleaned Data:
 Rank                          Song Artist Streams
    1                    The Weeknd  5.258  292019
    2                    Ed Sheeran  4.750   62017
    3             The Neighbourhood  4.351   32012
    4        The WeekndandDaft Punk  4.340  212016
    5                  Harry Styles  4.226   12022
    6                 Lewis Capaldi  4.210   82018
    7        Post MaloneandSwae Lee  4.105  182018
    8        DrakewithWizkidandKyla  4.010   52016
    9                    Ed Sheeran  3.811   32017
   10 The Kid LaroiandJustin Bieber  3.786   92021


✓ Successfully imported 10 songs into SQLite database

Database: spotify_songs.db

Top 10 Spotify Songs (from database):
--------------------------------------------------------------------------------
 1. The Weeknd                               | 5.258                |      292,019
 2. Ed Sheeran                               | 4.750                |       62,017
 3. The Neighbourhood                        | 4.351   